### Set up an evaluation script with a random classifier #8

Create a script that:

- Loads the dataset from `Initialize a dataset to evaluate the detection pipeline` #6
- Loads the predictions from a random classifier to classify claims using the taxonomy from `Define our contrarian claims taxonomy` #7
- Generates a text classification report
- Generates a confusion matrix plot

In [5]:
import pandas as pd
from pydantic import BaseModel
from typing import List
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
class EvaluateClassifier:

    def __init__(self, classifier):
        """
        Initialize the class with a fitted classifier passed as an argument.
        :param classifier: A classification model taking a serie of claims as input 
        """
        self.classifier = classifier
        self._validate_classifier_input()

    def _validate_classifier_input(self):
        """
        Validate that the classifier's `predict` method can accept a list of strings as input.
        """
        try:
            # Sample input to test if the classifier accepts a list of strings
            sample_input = ["sample claim 1", "sample claim 2"]
            self.classifier.predict(sample_input)
        except Exception as e:
            raise ValueError("The classifier's `predict` method must accept a list of strings as input. "
                             "Ensure the classifier is compatible with text data.") from e
    
    def load_data(self, file_path):
        """
        Load data from a csv file.

        :param file_path: Path to the Excel file containing the data
        :return: DataFrame with the loaded data
        """
        benchmark = pd.read_csv(file_path, sheet_name="benchmark")
        if 'claim' not in benchmark.columns or 'label' not in benchmark.columns:
            raise ValueError("Columns 'claim' and 'label' must be present in the benchmark")
        return benchmark
    
    def predict(self, benchmark):
        """
        Predict classes on the benchmark.

        :param X_test: Test features
        :return: Classifier predictions
        """
        claims = benchmark['claim']
        return self.classifier.predict(claims)
    
    def generate_classification_report(self, benchmark, y_pred):
        """
        Generate a classification report.

        :param y_test: True labels
        :param y_pred: Predictions
        :return: DataFrame of the classification report
        """
        report = classification_report(benchmark['label'], y_pred, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        print("Classification Report:\n", report_df)
        return report_df

    def plot_confusion_matrix(self, y, y_pred, labels):
        """
        Generate and display a confusion matrix.

        :param y_test: True labels
        :param y_pred: Predictions
        :param categories: List of classification categories
        """
        cm = confusion_matrix(y, y_pred, labels=labels)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
        plt.xlabel("Predicted Categories")
        plt.ylabel("True Categories")
        plt.title("Confusion Matrix")
        plt.show()

    def evaluate(self, file_path):
        """
        Run the complete evaluation: loading benchmark, predicting  with classifier, and evaluating.

        :param file_path: Path to the csv file containing the benchmark
        """
        # Load the data
        benchmark = self.load_data(file_path)
        y = benchmark['label']

        # Predict
        y_pred = self.predict(benchmark)
        labels = self.unique(self.y)

        # Generate the classification report
        self.generate_classification_report(y, y_pred)

        # Generate and display the confusion matrix
        self.plot_confusion_matrix(y, y_pred, labels)


### Test with a random classifier

In [3]:
benchmark_path = "../data/benchmark/cards_sample_1000.csv"

In [7]:
import os
from groq import Groq

import numpy as np
import json

## !!!!!to hide!!!!!!
os.environ["GROQ_API_KEY"] = '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'
LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

class Claim(BaseModel):
    disinformation_score: str
    classification: str
    
class Claims(BaseModel):
    claims: list[Claim]

class Classifier():

    def __init__(self):
        self.client = Groq()

    def assistant(content: str):
        return { "role": "assistant", "content": content }

    def user(content: str):
        return { "role": "user", "content": content }

    def chat_completion(
        self,
        messages: List[Dict],
        model: str = DEFAULT_MODEL,
        temperature: float = 0.6,
        top_p: float = 0.9,
    ) -> str:
        response = self.client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature,
            top_p=top_p,
        )
        return json.loads(response.choices[0].message.content)

    def completion(
        self,
        prompt: str,
        model: str = DEFAULT_MODEL,
        temperature: float = 0.6,
        top_p: float = 0.9,
    ) -> str:
        return self.chat_completion(
            [self.user(prompt)],
            model=model,
            temperature=temperature,
            top_p=top_p,
        )

    def classifier(
        self,
        prompt: str, 
        model: str = DEFAULT_MODEL
        )-> str:
        response = self.completion(prompt, model)
        label = response[0]['classification']
        return label

    def predict(self, row):
        prompt = f"""
            Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. Je vais te donner un extrait d'une retranscription de 2 minutes d'un flux TV ou Radio. 
            A partir de cet extrait liste moi tous les faits/opinions environnementaux (claim) uniques qu'il faudrait factchecker. Et pour chaque claim, donne une première analyse si c'est de la désinformation ou non, un score si c'est de la désinformation, ainsi qu'une catégorisation de cette allégation.
            Ne sélectionne que les claims sur les thématiques environnementales (changement climatique, transition écologique, énergie, biodiversité, pollution, pesticides, ressources (eau, minéraux, ..) et pas sur les thématiques sociales et/ou économiques
            Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois les champs suivants : 

            - "disinformation_score" - le score de désinformation (voir plus bas)
            - "classification" - la classification du type de désinformation suivant la taxonomie CARDS (voir plus bas)

            Pour les scores "disinformation_score"
            - "very low" = pas de problème, l'allégation n'est pas trompeuse ou à risque. pas besoin d'investiguer plus loin
            - "low" = allégation qui nécessiterait une vérification et une interrogation, mais sur un sujet peu important et significatif dans le contexte des enjeux écologiques (exemple : les tondeuses à gazon, 
            - "medium" = allégation problématique sur un sujet écologique important (scientifique, impacts, élections, politique, transport, agriculture, énergie, alimentation, démocratie ...) , qui nécessiterait vraiment d'être vérifiée, déconstruite, débunkée et interrogée. En particulier pour les opinions fallacieuses
            - "high" = allégation grave, en particulier si elle nie le consensus scientifique

            Pour la "classification":
            - "0" = "disinformation_score" est "very low" ou "low" 
            - "1": "climate change is not happening"
            - "2": "climate change is not caused by humans"
            - "3": "the consequences of climate change are not bad"
            - "4": "the solutions won't work"
            - "5": "climate science is unreliable"

            <transcription>
            {row['claim']}
            </transcription>
                """
        try:
            return self.classifier(prompt)
        except:
            return 'error'

NameError: name 'Dict' is not defined

In [ ]:
classifications = benchmark.apply(lambda row: class_from_prompt(row), axis=1)